In [1]:
import tensorflow as tf
import math

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
x=mnist.train.images
y=mnist.train.labels

In [4]:
mnist.test.images.shape

(10000, 784)

In [5]:
print (x.shape)
print (y.shape)

(55000, 784)
(55000, 10)


In [6]:
X = tf.placeholder("float", [None,784])
Y_=tf.placeholder("float",[None,10])

In [7]:
def batchnorm(Ylogits,beta):
    #exp_moving_avg = tf.train.ExponentialMovingAverage(0.998, iteration)
    bnepsilon = 1e-5
    mean, variance = tf.nn.moments(Ylogits, [0])
    BN = tf.nn.batch_normalization(Ylogits,mean,variance,beta,None,bnepsilon)
    return BN

In [8]:
# five layers and their number of neurons (tha last layer has 10 softmax neurons)
L = 200
M = 100
N = 60
O = 30

In [13]:
# Weights initialised with small random values between -0.2 and +0.2
# When using RELUs, make sure biases are initialised with small *positive* values for example 0.1 = tf.ones([K])/10
W1 = tf.Variable(tf.truncated_normal([784, L], stddev=0.1))  # 784 = 28 * 28
B1 = tf.Variable(tf.ones([L])/10)
W2 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
B2 = tf.Variable(tf.ones([M])/10)
W3 = tf.Variable(tf.truncated_normal([M, N], stddev=0.1))
B3 = tf.Variable(tf.ones([N])/10)
W4 = tf.Variable(tf.truncated_normal([N, O], stddev=0.1))
B4 = tf.Variable(tf.ones([O])/10)
W5 = tf.Variable(tf.truncated_normal([O, 10], stddev=0.1))
B5 = tf.Variable(tf.ones(10)/10)


In [14]:
Y1 = (tf.matmul(X, W1))
BN1=batchnorm(Y1,B1)
l1_BN = tf.nn.relu(BN1)

Y2 = (tf.matmul(l1_BN, W2))
BN2=batchnorm(Y2,B2)
l2_BN = tf.nn.relu(BN2)

Y3 = (tf.matmul(l2_BN, W3))
BN3=batchnorm(Y3,B3)
l3_BN = tf.nn.relu(BN3)

Y4 = (tf.matmul(l3_BN, W4))
BN4=batchnorm(Y4,B4)
l4_BN = tf.nn.relu(BN4)

Ylogits_BN = tf.matmul(l4_BN, W5) + B5
Ylogits_BN = tf.nn.softmax(Ylogits_BN)

In [15]:
cost = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits_BN, labels=Y_)
cost = tf.reduce_mean(cost)

In [16]:
correct_prediction = tf.equal(tf.argmax(Ylogits_BN, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [17]:
# variable learning rate
lr = tf.placeholder(tf.float32)

In [18]:
# training step, the learning rate is a placeholder
optimizer = tf.train.AdamOptimizer(lr).minimize(cost)

In [19]:
# learning rate decay
max_learning_rate = 0.003
min_learning_rate = 0.0001
decay_speed = 2000.0 # 0.003-0.0001-2000=>0.9826 done in 5000 iterations

In [20]:
training_epochs = 25
batch_size = 100

In [21]:
init =tf.global_variables_initializer()

In [22]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        c=0.0
        avg_cost=0.0
        # Loop over all batches
        for i in range(total_batch):
            
            learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)
            
            batch_X, batch_Y = mnist.train.next_batch(batch_size)
            #print(batch_X.shape)
            _,c=sess.run([optimizer,cost], feed_dict={X:batch_X,Y_:batch_Y,lr: learning_rate})
            avg_cost += c / total_batch

        print("epchos {} cross entropy={}".format(epoch+1,avg_cost))
            
    print ("Optimization Finished!")

    # Test model
    #correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print( "Accuracy:", accuracy.eval({X: mnist.test.images, Y_: mnist.test.labels}))

epchos 1 cross entropy=1.5829910193790082
epchos 2 cross entropy=1.5037690730528384
epchos 3 cross entropy=1.494472715421156
epchos 4 cross entropy=1.4876962837305927
epchos 5 cross entropy=1.48434135241942
epchos 6 cross entropy=1.4823183126883066
epchos 7 cross entropy=1.4800633194229817
epchos 8 cross entropy=1.4782088232040398
epchos 9 cross entropy=1.4765326523780826
epchos 10 cross entropy=1.4766314263777278
epchos 11 cross entropy=1.475000967979431
epchos 12 cross entropy=1.4737001991271967
epchos 13 cross entropy=1.4729909907687793
epchos 14 cross entropy=1.4723038868470624
epchos 15 cross entropy=1.472010118527845
epchos 16 cross entropy=1.4714870658787822
epchos 17 cross entropy=1.4717467641830468
epchos 18 cross entropy=1.4707135443253936
epchos 19 cross entropy=1.4703780924190166
epchos 20 cross entropy=1.4696327942067924
epchos 21 cross entropy=1.4694251255555604
epchos 22 cross entropy=1.469631668871099
epchos 23 cross entropy=1.4700495650551533
epchos 24 cross entropy=1.